In [17]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [18]:
baseline = pd.read_csv('../../data/baseline_features.csv')
news = pd.read_csv('../../data/news_features.csv')
baseline.head(1)

,source,headline,headline_processed,length,unique,frequency,class
0,Reuters,Germany's Kuehne examines offer for Signa's Ha...,germani kuehn examin offer signa hamburg skysc...,2.397895,0.693147,0.0,0


In [19]:
# declare feature (X), target (y) variables
X = baseline.drop(columns=['source','headline','headline_processed','class'])
y = baseline['class']

In [20]:
# split the baseline data into train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18)

In [22]:
xgbc = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

In [23]:
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
# predict test set
y_pred = xgbc.predict(X_test)

In [25]:
# evaluate model using accuracy, precision, recall
accuracy = accuracy_score(y_test, y_pred) 
conf_matrix = confusion_matrix(y_test, y_pred) 
classification_rep = classification_report(y_test, y_pred) 
print("Accuracy:", accuracy) 
print("Confusion Matrix:\n", conf_matrix) 
print("Classification Report:\n", classification_rep)

Accuracy: 0.6551724137931034
Confusion Matrix:
 [[ 0 20]
 [ 0 38]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.66      1.00      0.79        38

    accuracy                           0.66        58
   macro avg       0.33      0.50      0.40        58
weighted avg       0.43      0.66      0.52        58

